##  YOLOV5 모델 라벨링 및 학습 예제

In [ ]:
# 구글 드라이브 마운트
from google.colab import drive
import os

drive.mount('/content/drive/')

In [ ]:
%cd /content/drive/MyDrive/dataset

!unzip -qq "/content/drive/MyDrive/dataset/YOLO실습파일.zip"

In [ ]:
# 작업경로변경
print('현재 작업 경로 :', os.getcwd())
os.chdir('/content/drive/MyDrive')
print('변경된 작업 경로 :', os.getcwd())

In [ ]:
# YOLOv5 모델 가져오기
!git clone https://github.com/ultralytics/yolov5

In [ ]:
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install opencv-python matplotlib pyyaml torch

import torch
import yaml
from IPython.display import Image, clear_output  # to display images
import cv2
import matplotlib.pyplot as plt

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

In [ ]:
# 데이터셋 경로 및 데이터셋의 yaml 파일 경로 지정
data_dir = 
data_yaml = 

In [ ]:
# 데이터셋 yaml 파일 확인
with open(data_yaml) as f:
    film = yaml.load(f, Loader=yaml.FullLoader)
    display(film)

# yaml 파일에 저장된 label 이름과 train, valid 파일 경로를 출력

## 모델 학습

In [ ]:
!python train.py --img 416 --batch 64 --epochs 50 --data {data_yaml} --weights yolov5s.pt --cache

In [ ]:
# 텐서보드 실행
%load_ext tensorboard
%tensorboard --logdir runs

In [69]:
# 테스트 이미지 경로
test_data_dir = film['test']

In [ ]:
!python detect.py --weights runs/train/exp/weights/best.pt --img 640 --conf 0.3 --source {test_data_dir}

## 학습 모델 이용 라벨링 이미지 테스트

In [ ]:
# 테스트 결과 확인해보기

import glob
from IPython.display import Image, display

if not os.path.exists('/content/drive/MyDrive/yolov5/runs/detect/exp2' + '/') :
  raise Exception('test_exp_num 오류.')

for imageName in glob.glob('/content/drive/MyDrive/yolov5/runs/detect/exp2' + '/*.jpg'):
    display(Image(filename=imageName))
    print("\n")

In [ ]:
import os
import yaml
import cv2
import matplotlib.pyplot as plt
from IPython.display import Image, display
import glob

# 바운딩 박스 그리기 함수
def draw_boxes(image, boxes, color, labels=None):
    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = map(int, box)
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        if labels:
            cv2.putText(image, labels[i], (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
    return image

def load_labels(label_file, width, height):
    with open(label_file, 'r') as file:
        labels = [list(map(float, line.split())) for line in file.readlines()]
    boxes = [convert_box_format(label, width, height) for label in labels]
    return boxes

def convert_box_format(box, width, height):
    x_center, y_center, w, h = box[1:5]
    x1 = (x_center - w / 2) * width
    y1 = (y_center - h / 2) * height
    x2 = (x_center + w / 2) * width
    y2 = (y_center + h / 2) * height
    return [x1, y1, x2, y2]

# 테스트 결과 시각화

label_dir = '/content/drive/MyDrive/dataset/test/labels'  # 실제 라벨 디렉토리로 대체하세요
image_dir = '/content/drive/MyDrive/yolov5/runs/detect/exp'  # 실제 이미지 디렉토리로 대체하세요

if not os.path.exists(image_dir):
    raise Exception('image_dir 오류.')

for image_path in glob.glob(image_dir + '/*.jpg'):
    #display(Image(filename=image_path))

    # 이미지 파일명 추출
    image_file = os.path.basename(image_path)
    image_file_no_ext = os.path.splitext(image_file)[0]

    # 이미지 로드
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    # Ground Truth 바운딩 박스 가져오기
    label_file = os.path.join(label_dir, f'{image_file_no_ext}.txt')
    if os.path.exists(label_file):
        gt_boxes = load_labels(label_file, width, height)
    else:
        gt_boxes = []

    # Ground Truth 바운딩 박스 그리기
    image_with_gt = draw_boxes(image.copy(), gt_boxes, (0, 255, 0))

    # 시각화
    #plt.figure(figsize=(10, 10))
    plt.title('Labeling Data')
    plt.imshow(cv2.cvtColor(image_with_gt, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()
    print("\n")